In [1]:
from pyomo.common.dependencies import (
    numpy as np, numpy_available,
    pandas as pd, pandas_available,
)

import pyomo.common.unittest as unittest

from pyomo.environ import *
from pyomo.dae import *

from itertools import permutations, product, combinations
import idaes

from fim_doe import *

In [2]:
from pyomo.opt import SolverFactory
ipopt_available = SolverFactory('ipopt').available()


In [3]:
#@unittest.skipIf(not ipopt_available, "The 'ipopt' command is not available")

In [17]:
class doe_object_Tester(unittest.TestCase):
    def setUP(self):
        import reactor_kinetics as reactor
        
        createmod = reactor.create_model
        disc = disc_for_measure
        t_control = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]

        # design variable and its control time set
        dv_pass = {'CA0': [0],'T': t_control}


        # Define measurement time points
        t_measure = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
        measure_pass = {'C':{'CA': t_measure, 'CB': t_measure, 'CC': t_measure}}
        measure_class =  Measurements(measure_pass)
        
        # Define parameter nominal value 
        parameter_dict = {'A1': 84.79085853498033, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}

        def generate_exp(t_set, CA0, T):  
            '''Generate experiments. 
            t_set: time control set for T.
            CA0: CA0 value
            T: A list of T 
            '''
            assert(len(t_set)==len(T)), 'T should have the same length as t_set'

            T_con_initial = {}
            for t, tim in enumerate(t_set):
                T_con_initial[tim] = T[t]

            dv_dict_overall = {'CA0': {0: CA0},'T': T_con_initial}
            return dv_dict_overall
        
        # empty prior
        prior_all = np.zeros((4,4))

        prior_pass=np.asarray(prior_all)

        exp1 = generate_exp(t_control, 5, [300, 300, 300, 300, 300, 300, 300, 300, 300])

        self.doe = DesignOfExperiments(parameter_dict, dv_pass,
                             measure_class, createmod,
                            prior_FIM=prior_pass, discretize_model=disc, args=args_)

    def test_sequential_finite(self): 


        result = self.doe.compute_FIM(exp1,mode='sequential_finite', FIM_store_name = 'dynamic.csv', 
                                        store_output = 'store_output', read_output=None,
                                        scale_nominal_param_value=True, formula='central')


        result.calculate_FIM(self.doe.design_values)

        self.assertAlmostEqual(np.log10(result.trace), 2.962954, places=3)
        self.assertAlmostEqual(result.FIM[0][1], 1.840604, places=3)
            
            
            
            

In [18]:
if __name__ == '__main__':
    unittest.main()

E
ERROR: /home/parallels/ (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/home/parallels/'

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


SystemExit: True

/home/parallels/anaconda3/envs/pyomo/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
